Rossmann Store Sales Örneği

Merhaba, bu çalışma sayfasında Rossmann Store Sales datası ile ilgili yeni bir çözüm örneğini paylaşıyorum. Konu hakkındaki yorumlarınız benim için önemlidir.

Bazı kısımlar da görüş eksikliği veya dolaylı yoldan gidilmiş olabilir yeni olan tüm önerilere açığım umarım bu örnek çalışma sizin için faydalı olur.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Rossmann Store Sales datası ile farklı 7 ülkede bulunan 1115 farklı dükkan için satış rakamları mevcuttur. Amacımız her farklı dükkanın belirli bir dönem sonunda ne kadarlık bir satış yapacağını tahmin etmektir.

Data içerisinde bulunan değişkenler ile ilgili bilgiler aşağıdaki gibidir:

Id --> İndex

Store --> Tekil mağaza numarası

Sales --> Günlük ciro

Customers --> Gün içerinde mağazaya gelen müşteri sayısı

Open --> Mağazanın o gün için açık veya kapalı olduğu bilgisi. (1 = açık, 0 = kapalı)

StateHoliday --> O gün için bulunduğu ülkenin tatil günü olup olmadığı ( a = resmi tatil, b = paskalya tatili, c = Yılbaşı, 0 = tatil değil)

SchoolHoliday --> Okul tatili.

StoreType --> a = basit ürünler, b = ekstra ürünler, c = en fazla çeşidin olduğu ürünler.

CompetitionDistance --> En yakın rakip mağazanın uzaklığı

CompetitionOpenSince(Month/Year) --> En yakın rakip mağazanın açılış yıl ve ayı.

Promo --> O günde mağazanın promosyon ürünü olup olmadığı.

Promo2 --> Mağazanın promosyona katılıp katılmadığı.

Promo2Since(Year/Week) --> Eğer Promo2 1 ise Promo2'ye başladığı yıl ve yılın haftası.

PromoInterval --> Promo2'nin başladığı aylar.

Data olarak elimizde 3 farklı kaynak var bunlardan ilki train datası. Train datasında günlük olarak her mağazanın satış, müşteri, tarih, tatil gibi bilgileri mevcut ikinci data test datası. Test datasıda tıpkı train datasındaki bilgileri barındırıyor (hedef değişken Sales hariç). Son olarak store datası ise rakip mağaza bilgileri, promosyon bilgileri gibi değişkenleri içeriyor ilk olarak datalarımızı yükleyelim.

In [ ]:
train = pd.read_csv("/kaggle/input/rossmann-store-sales/train.csv")
test = pd.read_csv("/kaggle/input/rossmann-store-sales/test.csv")
store = pd.read_csv("/kaggle/input/rossmann-store-sales/store.csv")

train.head()

Train ve store datasını birleştirmek analizi tek bir data üzerinden yapmamızı sağlayacak aynı şekilde şimdiden test datasına eklemek tahmin işleminde kolaylık sağlayacaktır.

In [ ]:
train = pd.merge(train, store, on = "Store", how = "left")
test_data = pd.merge(test, store, on = "Store", how = "left")

train.head()

Data hazırlandı şimdi data üzerinden çeşitli kırılımlar bulmaya çalışabiliriz.

Önce satış rakamının aylara göre nasıl davrandığını görmek istiyorum. Satışlarda belki bir çeşit mevsimsellik olabilir (yazları veya kışları satışların artması azalması gibi) Aslında Rossmann bir ilaç mağazasıdır bu sebeple bizim satışlarımız çoğunlukla medikal ürünler olacaktır yani hap, şurup, vitaminler vs gibi bu sebeple kış ayları lehine bir mevsimsellik olabilir.

In [ ]:
date = train["Date"]
date = pd.DataFrame(date)
date["Date"] = pd.to_datetime(date["Date"])

y = []
i = 0
while i < 1017210:
    x = date["Date"][i].month
    y.append(x)
    i += 1


In [ ]:
y = pd.DataFrame(y, columns = ["Month"])
y["Month"].value_counts()

train["Month"] = y["Month"]

y = []
i = 0
while i < 1017210:
    x = date["Date"][i].year
    y.append(x)
    i += 1

In [ ]:
y = pd.DataFrame(y, columns = ["Year"])
y["Year"].value_counts()
train["Year"] = y["Year"]

y = []
i = 0
while i < 1017210:
    x = date["Date"][i].day
    y.append(x)
    i += 1

In [ ]:
y = pd.DataFrame(y, columns = ["Day"])
train["Day"] = y["Day"]
aylar = pd.pivot_table(train, values='Sales', index = ["Month"],
                        aggfunc=np.sum)

aylar["Month"] = [1,2,3,4,5,6,7,8,9,10,11,12]

import seaborn as sns

sns.barplot(y = "Sales", x = "Month", data = aylar)

En fazla satışın olduğu ay mart olarak gözükmekte buna karşın temmuz ayından sonra satış rakamları çok ciddi bir şekilde düşmüş gözüküyor. Bunun nedenini anlamak için herhangi bir mağazayı seçip sadece o mağazanın bilgileri incelenebilir.

In [ ]:
ornek_magaza = train.loc[train["Store"] == 13]

In [ ]:
x = ornek_magaza.loc[ornek_magaza["Month"] == 3].sum()
y = ornek_magaza.loc[ornek_magaza["Month"] == 11].sum()

print("=======================mart ayı=======================", x, "\n", "=======================kasım ayı=======================",y)

Yukarıdaki çıktıda aynı mağazanın mart ve kasım ayları tüm bilgileri var satış rakamı mart ayı için 384.621 iken kasım ayı için 122.826 yani yukarıdaki grafik gibi. Bunun nedeni olarak en belirgin özellik "open" değişkeni olarak gözüküyor mart ayında toplamda (yıllar için) açık olma gün sayısı 77 iken kasım ayında ise bu sayı 25'tir. Yani satış adedinin azlığı mağazanın kapalı olmasından kaynaklanmaktadır.

Eğer sadece açık mağazalar ile aynı grafiği çizdirirsek daha gerçek sonuçlara ulaşabiliriz. Aynı zamanda satış tahminleri içinde açık mağazaları göz önüne almak daha iyi olacaktır çünkü eğer mağaza kapalı ise satışı otomatik olarak 0 olacaktır.

In [ ]:
magaza = pd.pivot_table(train, values = "Open", index = ["Store"], aggfunc = np.sum)
max(magaza["Open"])

Yukarıda yaptığımız işlem açık gün sayılarını gösteren mağazaların pivotu. En fazla açık olma gün sayısı ise 942. Burada yapmak istediğim şey eşit sayıda açık gün sayısı olan mağazalara bakıp yeni grafik çizdirmek bunun içinde tatiller hariç sürekli açık olan mağazaları seçmek daha doğru olabilir. Bu mağazaların açık gün sayıları ise 942 olarak gözükmekte. Şimdi açık gün sayısı 942 oln mağazaları seçip bunların aylara ve günlere göre grafiklerini çizelim.

In [ ]:
magaza = magaza.loc[magaza["Open"] == 942]

hep_acik = pd.DataFrame()

for i in magaza.index:
    magazalar = train.loc[train["Store"] == i]
    hep_acik = pd.concat([hep_acik, magazalar], axis = 0)

aylar = pd.pivot_table(hep_acik, values = "Sales", index = ["Month"], aggfunc = np.sum)

aylar["Month"] = [1,2,3,4,5,6,7,8,9,10,11,12]

sns.barplot(x = "Month", y = "Sales", data = hep_acik)

Evet artık aylara göre daha gerçek sonuçlar elde ettik. Burada gerçekten de kış aylarında satışların daha fazla arttığı gözükmekte yani bir bakıma mevsimsellik bulunmakta. Şimdi günlere göre aynı mağazaların satış rakamlarını bulalım.

In [ ]:
günler = pd.pivot_table(hep_acik, values = "Sales", index = ["DayOfWeek"], aggfunc = np.sum)

günler["Day"] = ["P.tesi", "Salı", "Çarşamba", "Perşembe", "Cuma", "C.tesi", "Pazar"]

sns.barplot(x = "Day", y = "Sales", data = günler)

Günlerde P.tesi, Cuma ve Pazar günleri satışların arttığı gözüküyor.

Şimdi ise her müşterinin o mağazaya ne kadar hesap ödediğini bulalım. Burada belki bazı günler gelen müşteri sayıları diğer günlerden fazla olabilir ama belkide bu müşteriler ortalama daha az ücret ödeyebilirler veya tam tersi gün içerisinde az sayıda müşteri gelip daha fazla ücret ödemiş olabilirler her gün için müşteri başına ortalama ödenen ücreti bularak dataya yeni bir değişken ekleyebiliriz.

In [ ]:
train = train.loc[train["Open"] == 1]
train["günlük_ort"] = train["Sales"] / train["Customers"]

train.head()

Şimdi ilgili mağazanın rakebet ettiği mağazanın ay sayısını bulalım bu satışlar üzerine etkisi olan bir değerdir. Rakip bir mağazanın açılışı satışları düşüren bir etkiye sahip.

In [ ]:
train["CompetitionMonths"] = 12 * (train.Year - train.CompetitionOpenSinceYear) + (train.Month - train.CompetitionOpenSinceMonth)
train["CompetitionMonths"] = train["CompetitionMonths"].replace(24187, 0)

eksiler = train.loc[train["CompetitionMonths"] > 24000]
train = train.loc[train["CompetitionMonths"] < 24000]

eksiler["CompetitionMonths"] = 0
train = pd.concat([train, eksiler], axis = 0)

In [ ]:
train.head()

Şimdi data da bulunan boş satırları 0 ile değiştirip data da bulunan harf değerleri sayısal değerlere dönüştürerek modellemeye hazır hale getirelim.

In [ ]:
train = train.replace(np.nan, 0)

del train["Date"]
del train["CompetitionOpenSinceMonth"]
del train["CompetitionOpenSinceYear"]
del train["Open"]

train.loc[train['StoreType'] == 'a', 'StoreType'] = '1'
train.loc[train['StoreType'] == 'b', 'StoreType'] = '2'
train.loc[train['StoreType'] == 'c', 'StoreType'] = '3'
train.loc[train['StoreType'] == 'd', 'StoreType'] = '4'
train['StoreType'] = train['StoreType'].astype(float)

train.loc[train['Assortment'] == 'a', 'Assortment'] = '1'
train.loc[train['Assortment'] == 'b', 'Assortment'] = '2'
train.loc[train['Assortment'] == 'c', 'Assortment'] = '3'
train['Assortment'] = train['Assortment'].astype(float)

train.loc[train['StateHoliday'] == 'a', 'StateHoliday'] = '1'
train.loc[train['StateHoliday'] == 'b', 'StateHoliday'] = '2'
train.loc[train['StateHoliday'] == 'c', 'StateHoliday'] = '3'
train['StateHoliday'] = train['StateHoliday'].astype(float)

train["PromoInterval"].value_counts()
train.loc[train['PromoInterval'] == 'Jan,Apr,Jul,Oct', 'PromoInterval'] = '1'
train.loc[train['PromoInterval'] == 'Feb,May,Aug,Nov', 'PromoInterval'] = '2'
train.loc[train['PromoInterval'] == 'Mar,Jun,Sept,Dec', 'PromoInterval'] = '3'
train['PromoInterval'] = train['PromoInterval'].astype(float)

train.isnull().sum()

In [ ]:
train.head()

Modelleme

In [ ]:
df = train.copy()
del df["Sales"]
y = pd.DataFrame(train["Sales"])
x = df


from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 42)

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(x_train, y_train)
xgb_tahmin = xgb.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
print("R^2 Skoru :", round(r2_score(y_test , xgb_tahmin), 4))
print("Hata Kareler Ortalaması: ", round(np.sqrt(mean_squared_error(y_test , xgb_tahmin))))

In [ ]:
print("Teşekkürler!")